# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [203]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny? 

Haszowanie samo w sobie nie uwzględnia kluczu prywatnego ani publicznego, jest to tak naprawdę przekształcenie danych wejściowych na stałej długości wyjściowej(skrót). Nie potwierdza, że dane te pochodzą od określonego nadawcy. Hasz może zostać obliczony przez każdego, kto zna dane wejściowe, więc nie ma sposobu na potwierdzenie, że nadawca faktycznie przesłał te dane.

2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?

Kody MAC, w szczególności popularny HMAC, nie weryfikują, kto i do czego użył klucza prywatnego. Klucz prywatny HMAC jest znany tylko nadawcy i odbiorcy, co oznacza, że tylko te osoby mają wiedzę na temat jego wykorzystania. Jeśli odbiorca lub atakujący zdobędzie klucz prywatny HMAC, może wykorzystać go do wygenerowania fałszywego kodu MAC i podszyć się pod nadawcę, bez możliwości jednoznacznego przypisania nadawcy do wiadomości.

3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

* Integralność oznacza, że podpis cyfrowy gwarantuje, że wiadomość nie została zmieniona w trakcie przesyłania. Podpis cyfrowy jest generowany na podstawie całej treści wiadomości oraz klucza prywatnego nadawcy, a odbiorca używa klucza publicznego nadawcy do weryfikacji, czy podpis cyfrowy jest poprawny. Jeśli podpis cyfrowy jest poprawny, odbiorca może mieć pewność, że treść wiadomości nie została zmieniona w trakcie przesyłania.

* Autentyczność oznacza, że podpis cyfrowy gwarantuje, że wiadomość została wysłana przez rzeczywistego nadawcę. Podpis cyfrowy jest generowany przy użyciu klucza prywatnego nadawcy, co oznacza, że tylko nadawca może wygenerować prawidłowy podpis. Odbiorca używa klucza publicznego nadawcy do weryfikacji podpisu, a jeśli podpis jest poprawny, odbiorca wie, że wiadomość została wysłana przez rzeczywistego nadawcę.

* Non-repudiation oznacza, że podpis cyfrowy uniemożliwia nadawcy zaprzeczenie, że wysłał daną wiadomość. Ponieważ podpis cyfrowy jest generowany przy użyciu klucza prywatnego nadawcy, tylko nadawca może go wygenerować. Odbiorca używa klucza publicznego nadawcy do weryfikacji podpisu, co pozwala jednoznacznie przypisać wiadomość do nadawcy. W ten sposób nadawca nie może zaprzeczyć, że wysłał daną wiadomość.

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [204]:
nlen = 2048

def generate_nums() :
    #stałe
    con = 65537
    N = 256
    
    a = pow(2,N)
    b = pow(2, nlen//2)
    c = pow(2, (nlen - 1)//2)
    d = pow(2, nlen//2 - 100)
   

    #krok 1 
    #warunek 3,4,5
    while True :
        p, q = cu.getStrongPrime(nlen//2), cu.getStrongPrime(nlen//2)
        if ( c<p ) & ( p<b ) & ( c<q ) &  ( q<b ) & (abs(p-q) > d) :
            break
    
    #krok 2 i 3
    n = p * q
    f = (p - 1) * (q - 1)

    #krok 4
    #warunek 1,2 
    while True :
        e = cu.getRandomRange(65537, min(f, a))
        if ( cu.GCD(f,e) == 1) & (e < f) & (math.lcm(p-1, q-1) >= e*b ):
            break
    
    #krok 5
    d = cu.inverse(e, f)

    return d, n, e

d, n, e = generate_nums()

print(f"""Klucz prywatny (d,n)\nd={d}\nn={n}\nKlucz publiczny (p,q)\ne={e}\nn={n} """)


Klucz prywatny (d,n)
d=3513730082479551140237288908531176085907896597770546757844003884535189915460694868676314938956978798847274240231196220503583595798024952930084143093887772611299681503231840855429946290338249116330250649914060113724999349900659150558434117008149549648919438312873706577117968556280118481988921235567116658335457869960147723128448109723261206967177994963731033899207492898698903238015808267016409542697895493033646347781298188059322687767454754784376032132271996950797838808736070413447541599254625893950393933371132252342904227544633363227444800443793935770303217250121296234879866731221336579827238721290296669440957
n=2160929873485933113232863475268204130922740257527936901120682036908943685777601481014293437610100082038351144575114684965359022311068745557977033035926602075008002196994520747175553598265988959651110168791489553834181545974141978430718554719513943082938408200952442849146300050776404989708569191025111294527380996392253095200932818828755118746353102622134405

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [205]:
d, n, e = generate_nums()

def encrypte(m) :
    #krok 1
    mBytes = m.encode("utf-8")
    mInt = int.from_bytes(mBytes, byteorder='big')
    #krok 2
    C = pow(mInt,e,n)
    return C.to_bytes(math.ceil(C.bit_length() / 8), byteorder='big')

M = "Ala ma kota"
C = encrypte(M)
print(C)

b"%\xd0\xd4\xe3\x13/\xebB\xa7\x13\xc90\\\x02\xd9\xcc\xa3\xd0q\xbf$\xab\xefs&\xed\x90\xec\xcd\r\xd4\xcd\xb3\xb0\x02X\xa0%\x9eH*z}\x0e\xca\xa8g\x11\xf9\xe3_\xa3&\x10`W\x03M\x8b\xd4%\xcc\x19^%\xf7\x86E\x0eg\xc7\x93\x84M\x81\x11\xbd5\xd2Y\xe6~\xcek\xa3\xd0\xba\x15\xc0\xd9&\x82^\xb4%\xff\xb7\xa4\xc3\xcb\x85\xac\x9e9U\x91t\xed\x19\x0e\xd4\xc1\xa3\xe0\xe2\x0b\x846\xeb\x8fi\xc6N3\xd8n\xd8w\xfd^\xb7\xf2\xc4P\x99\x14\xca\x91)\xe9H\xbe\x929\xce\x88\xecz\xe72OX\xcb]\x08\x14\xf1M[\r\xcf\r.O\x89\x1c\x10\xa0\xfb7\x05\xc2\x99\x94N\xf5\xa5\x9d\x0bJ\xb4\x02 \xa1!\xf3D\xd2:\xfcv\xe4\xc3\x94\xaa\xdaI7\xba\xe4$]3z\xa4\x0e\x9c=\xc4k\xbbc&vlb\xe2\x87\\Z<J\x97'\x17\xe6g\xea\x19_B\x81nc)\x92\x9e\xb6G\xfb\x04\xbc\x99\x8cQ\xdbn@`W\xc7\xb8\x0b\x7f\xb4\xbd"


In [206]:
def decrypte(c) :
    #krok 1
    M = pow(int.from_bytes(c, byteorder='big'),d,n)
    #krok 2
    mBytes = M.to_bytes((M.bit_length() + 7) // 8, byteorder="big")#zamiana intow na bajty, gdzie 1 bajt=8bitow
    m = mBytes.decode("utf-8")
    return m
decrypte(C)

'Ala ma kota'

Dodawanie 7 jest używane, aby zapewnić poprawne zaokrąglenie. Np. jeśli liczba bitów, które muszą zostać zapisane, to 10 to jeśli podzielimy tę wartość przez 8, to otrzymamy 1 bajt i resztę, która wynosi 2 bity. Bez dodania wartości 7, zaokrąglilibyśmy wynik do 1 bajtu, co spowodowałoby utratę 2 bitów. Dodając 7, uzyskujemy 17, co dzieląc przez 8 bez reszty daje nam 2 bajty. W ten sposób zapewniamy, że nie tracimy żadnych bitów.

In [207]:
#drugi sposób za pomocą wbudowanych funkcji
m = cu.bytes_to_long(M.encode())

c = pow(m, e, n)
m = pow(c, d, n)

m = cu.long_to_bytes(m).decode("utf-8")
print(m)

Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [208]:
def create_signature(m) :
    h = hl.sha256(m.encode()).digest()
    h = cu.bytes_to_long(h)
    SIG = pow(h,d,n)
    return M, SIG,h

M,SIG,h = create_signature(M)
print(f"""M = {M}\nSIG = {SIG}""")


M = Ala ma kota
SIG = 4558049912801924204729666342515598763200690862264645759322552870831622777357985573502233195268522493308615182832859337839316251985036158932119819499842175713893873604446824608850454813853127797911625521031940397275839196348710433163206220800566349455863249094667998172256695512095946497804172366946583840933797002221013891748681530339976835935591660015313573226081519752661580647242713950778697981980936929741698963180271451885762178958563070267147432238151447659080121795704401418222998744905624378866663373863331807132407601316176977481708528161753686473127927714499760248594985974065555159614789519712523727240924


In [209]:
def verify_signature(SIG,h) :
    VER = pow(SIG,e,n)
    if VER == h :
        print("Successfull verification") 
    else :
        print("Verification failed")

M = "Ala ma kota"
M,SIG,h = create_signature(M)
verify_signature(SIG,h)

Successfull verification


In [210]:
verify_signature(SIG-1,h)

Verification failed


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [211]:
hlen = 256 // 8 #256 bitow/8 = 32 bajty

def mgf1(m, l) :
    if isinstance(m, str):
        m = m.encode()

    output = bytearray()
    for i in range(0, math.ceil(l/hlen)) :
        iByte = i.to_bytes(4, byteorder='big')
        tmp = hl.sha256(m + iByte).digest()
        output += tmp

    return output[:l+1]

mgf1(b"gg", 20)


bytearray(b'\xc8b\xdf\x19\xd0p\xf8c\x1f\x13\xc9{\x96\xe4\xfd\x02\xa3\xa7w\xeb\x16')

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [212]:
import Crypto.Random as cr

In [213]:
M = "Ala ma kota"
hlen = 256 // 8

def oaep_encode(k, m):

    if isinstance(m, str):
        m = m.encode()
        
    mlen = len(m)

    if mlen > (k - 2 * hlen - 2):
        raise ValueError("Error mlen > k - 2 * hlen - 2")
    
    lHash = hl.sha256(b"").digest()

    ps = bytearray(k - mlen - 2 * hlen - 2)

    db = lHash + ps + b'\x01' + m

    if len(db) != (k - hlen - 1) :
        raise ValueError("Error len(db) != (k - hlen - 1)")

    seed = cr.get_random_bytes(hlen)

    dbMask = mgf1(seed, k - hlen - 1)

    maskedDB = bytearray([db[i] ^ dbMask[i] for i in range(len(db))])

    seedMask = mgf1(maskedDB, hlen)

    maskedSeed = bytearray([seed[i] ^ seedMask[i] for i in range(len(seed))])

    em = b'\x00' + maskedSeed + maskedDB

    if len(em) != k :
        raise ValueError("Error len(em) != k")

    return em



In [214]:
k = 100
M= 'Ala ma kota'
message_encoded = oaep_encode(k, M)
message_encoded

b'\x00\xf0P\xacBm\x16\x14\x97\x17\xde=\xf6$}iR\x9f\xbc\xc3\x93\x06\x95*q\xb6\x18\x14\xb3V\x82\x03oI3\x9a\x077\xf9\x86\x94Y\x7f\xfe\xfd=\x8a\x9a\x91{\xe4\xfe\x85\xf6\x0f\xfcI\xaa\xac7\xf2\xdb3\xb0\x04\xa5\xf9\x06w\xa7\xbdm\x13\xa4!d\xf0 \xbeMK\x00\x14Uzq\xd8\xd3\xecd-\xb5\xa4^\xa3\xe0Z\xb1j\x14'

In [215]:
def oaep_decode(k, em):

    y = em[0]
    
    if y != 0:
        raise ValueError("Error y != 0")
    
    maskedSeed = em[1:hlen+1]
    maskedDB = em[hlen+1:]

    lHash = hl.sha256(b"").digest()

    seedMask = mgf1(maskedDB, hlen)

    seed = bytearray([maskedSeed[i] ^ seedMask[i] for i in range(len(maskedSeed))])

    dbMask = mgf1(seed, k - hlen - 1)

    db = bytearray([maskedDB[i] ^ dbMask[i] for i in range(len(maskedDB))])
    
    lHashPrim = db[:hlen]
    db = db[hlen:] #kolejne bajty

    i = 0
    while db[i] != 1: 
        i += 1
    
    if db[i] == 1: 
        i += 1
    
    m = db[i:]
    
    if lHash != lHashPrim:
        raise ValueError("Error lHash != lHashPrim")
    return m

In [216]:
message_decoded = oaep_decode(k, message_encoded).decode()
print(message_decoded)

Ala ma kota


In [218]:
def oaep_encrypte(m, e, n) :
    k = math.ceil(n.bit_length() / 8)
    mBytes = oaep_encode(k, m)
    mInt = int.from_bytes(mBytes, byteorder='big')
    C = pow(mInt,e,n).to_bytes(k, byteorder='big')
    return C

M = "Ala ma kota"
c = oaep_encrypte(M, e, n)
print(f"Encryption: {c}")

Encryption: b'\r\xa8\x92\xba\x91\xf4\xe2v\xec\xd1\x02i\xda\\e\x1d\rv\x95\xadm\x05\xdb\x10\xf8\x10\xda\xd8\xac\xcf\x1bb\xd8vh\xad\xa6\xfc\t\xd8\x9f(1\xd4\x06\xe2:\xe8~\x80w\xdd?k\x01\xcc\xc4\x15\x14 \x88\xfat%\x1f\x88\xc3\xac\xb3\x1a\xf5\xb9i\xb5\xf4VkK\neqi\xdd\xa3\xb2\x94\x86W(\xc3\xe1\xb7-\xb3{\x1cd\xc5<lr\x16\x1c\xd0\x00 \x82o\xe0\xf2\xf2\xa5\x8e}.\xe9\xd4\xfd\xdb\xd8\xe0f4E\xbb/\x9c\x06\xce\x0bs\xab[\xca\xb8\x9e\xfc\xb6\xa7:\xa2\xd5\x11\x92\x11 \xb4\xb2\xd2eQ\x1f\xbc\x009\x1a\x7f\x81\xe4@\x0e\x84g\xdao\xf1+\xe8\x8a7\xea\xdc\xf7]\x1e2~\x94o\xc3\xb2\x12\x994c\xef2\x0e\xd6c\xc6\x9a\xc1\xc1\x97RN\xe4\x0cV\xa7\xb5\x0fT\x86us\xdb\x14\x81\xdc\x13\xcb<\xc0-Dz\xb7\x81\r\xa1#\x882~[d\x9aU\x1a\xac2*\x89\xcfI#\xdf\xd6T2k\xc1A\xff\xc4Ts\x8608\xf9w\x03\xb8'


In [219]:
def oaep_decrypte(c, d, n) :
    k = math.ceil(n.bit_length() / 8)
    M = pow(int.from_bytes(c, byteorder='big'),d,n)
    mBytes = M.to_bytes(k, byteorder="big")#zamiana intow na bajty, gdzie 1 bajt=8bitow
    m = oaep_decode(k, mBytes)
    return m

print(f"Decryption: {oaep_decrypte(c, d, n)}")


Decryption: bytearray(b'Ala ma kota')


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [220]:
M = b"Ala ma podpis"
slen = hlen
em_bits = nlen - 1
emlen = nlen//8

def emsa_encode(m):

    m_hash = hl.sha256(m).digest()
    if emlen < hlen + slen + 2:
        raise ValueError('Error emlen < hlen + slen + 2')
    
    salt = cr.get_random_bytes(slen)
    
    m_prime = b"\x00"*8 + m_hash + salt
    m_hash_p = hl.sha256(m_prime).digest()
    
    ps = bytearray(emlen - slen - hlen - 2)
    db = ps + b"\x01" + salt
    
    db_mask = mgf1(m_hash_p, emlen - hlen - 1)

    masked_db = bytearray([db[i] ^ db_mask[i] for i in range(len(db))])

    masked_db[:8 * emlen - em_bits] = b'\x00' * (8 * emlen - em_bits)
    em = masked_db + m_hash_p + b'\xbc'
    
    return em
signature = emsa_encode(M)
print(f"""Signature: {signature}""")

Signature: bytearray(b'\x00\xb4?\xeeV\xb5Zl\xd2\xc3\x02\x80\xd8\x95\x96Bi\xc032\xb5\t\x89\xa9u\xfe\x1d\x03..*\xda\xc6\xb4R\xe5\r\x0e\x92\xabd\x8f\x0cR\xf6\x97\xe3\xcc\xfa&p/w\x99\xab\xc1k"\x05\x7fk$\xdd=\xf1\x10V&&\xf2\xe9H\x90y\xe7o\n\xc9\x95]^\xb1\xd0\xea\xcc\xf42\x97\xa4\x7f\xe6\xb7\xef\r\r&u\x8e\xda\xa9U\xadB\x97\xadx\xd2\x1e\x80\xe0jO\xbbeE\xd9\x92D\xab0\x02\x1c\xfc\x84\x0b\x01E\x06I=\xdd\xd2\x96\xdb\xa5N\x8c\x08b\xd34\xf3\x03\x02\xcdZ\x85I\xc2\xcd\xf6\xfd\x83\x92\x8d\xde\xae\x13o8E\'^\xa9@\x9cv\x0b\xdc\x9a\xad\xbeL\xd6\x8fl\ra\x19\xe6ce\xca\xeaX\xab\xb2\x7f\xcf\xf7\x82\xb5=`e=N\x18C\xe7J\x00\xc1\xf4\xda=7=q\n\\;\x0ea\xef\xd4\xd6\xe0\x01\xe3\xa57t\xfb\x03\xcf\xb1\x1f0\xb5\xe9\xdd(8Z\xf9\x93u\x9cPF\xbf\xe8T\x89\x9f\x07\xcc@\xdb\x11\x7fu\xc7\x84\xbc')


In [221]:
def emsa_decode(em, m):

    m_hash = hl.sha256(m).digest()
    if emlen < hlen + slen + 2:
        raise ValueError('Error: emlen < hlen + slen + 2')
    
    if em[-1] != 0xbc:
        return('Error: Verification failed 1')
    
    masked_db = em[:emlen - hlen - 1]
    em = em[emlen - hlen - 1:]
    
    m_hash_prime = em[:hlen]
    em = em[hlen:]

    if masked_db[:8 * emlen - em_bits] != b'\x00' * (8 * emlen - em_bits):
        return('Error: Verification failed 2')
    
    db_mask = mgf1(m_hash_prime, emlen - hlen - 1)

    db = bytearray([masked_db[i] ^ db_mask[i] for i in range(len(masked_db))])

    db[: 8 * emlen - em_bits] = b'\x00' * (8 * emlen - em_bits)

    if db[:emlen - hlen - slen - 2] != b'\x00' * (emlen - hlen - slen - 2) or db[emlen - hlen - slen - 2] != 1:
        return('Error: Verification failed 3')
    
    salt = db[-slen:]

    m_prime = b'\x00' * 8 + m_hash + salt

    m_hash_prime2 = hl.sha256(m_prime).digest()

    if(m_hash_prime != m_hash_prime2):
        return('Error: Verification failed 4')
    
    return('Successfull verification!')

In [222]:
signature_decoded = emsa_decode(signature, M)
print(f"""Signature: {signature_decoded}""")

Signature: Successfull verification!


In [223]:
def create_signature_emsa(m) :
    h = emsa_encode(m)
    h = cu.bytes_to_long(h)
    SIG = pow(h,d,n)
    return M, SIG,h

M = b"Ala ma podpis"
M,SIG,h = create_signature_emsa(M)
print(f"""M = {M}\nSIG = {SIG}""")

M = b'Ala ma podpis'
SIG = 4034641175745043300367801028540067845424324172619701322597859295669752835050093856048165437053688440359175490153886491801506780586086523979550025514514507634514754673018206173105833821631920951066675196976523232824399941824689992218362883435064412142951148709773112577868907343573359935336026406774228956480698541317189065869827765423994720084873050524689153366574792785029634686896101369328239684116676838822065822748290602527938213713300945302202750084920806785491951943104217947096984215058545104634705095119874615799212469156110433587817185930184971763754697729263578718401395689563999283305744093556117679899225


In [224]:
def verify_signature_emsa(SIG,h) :
    VER = pow(SIG,e,n)
    h = emsa_decode(h)
    if VER == h :
        print("Successfull verification") 
    else :
        print("Verification failed")

verify_signature(SIG,h)

Successfull verification


In [225]:
verify_signature(SIG-1,h)

Verification failed


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [226]:
def encrypt_kem(e,n):
    rand = cr.get_random_bytes(1)
    key = hl.sha256(rand).digest()
    ciphered_key = oaep_encrypte(key,  e, n)
    return ciphered_key, key

def decrypt_kem(ciphered_key, d, n):
    key = oaep_decrypte(ciphered_key,  d, n)
    return key

ciphered_key, key = encrypt_kem(e,n)
print(key)
print(decrypt_kem(ciphered_key, d, n))

b'5\xaf-\x15\xeb\xdeMg\xb2\x95i\xec\x19t\x9e\x81<v\xf0"\x86\xaf\xc5\xa2\xa9\x87\x1c+\xfb\xda\xab2'
bytearray(b'5\xaf-\x15\xeb\xdeMg\xb2\x95i\xec\x19t\x9e\x81<v\xf0"\x86\xaf\xc5\xa2\xa9\x87\x1c+\xfb\xda\xab2')
